# 📝 TextCNN Sentiment Analysis

<details>
<summary><strong>📋 About This Project & Objectives</strong></summary>

This exercise implements the TextCNN sentiment analysis, a classic case in the deep learning field. This project demonstrates advanced natural language processing techniques using convolutional neural networks for text classification.

**Learning Objectives:**

- 📝 Master text preprocessing and tokenization techniques for sentiment analysis
- 🏗️ Build and configure TextCNN architecture with multiple filter sizes
- ⚙️ Implement text classification training strategies with embeddings

**Project Workflow:**
- ⚫ Download the required dataset (rt-polarity dataset)
- ⚫ Define data preprocessing function and generate data for training and validation
- ⚫ Define the TextCNN model structure build, training, validation, offline model loading, and online inference functions
- ⚫ Define various parameters required for training, such as optimizer, loss function, checkpoint, and time monitor
- ⚫ Load the dataset and perform training
- ⚫ After training completion, use the test set for validation

</details>

## 📚 Imports and Environment Setup
Setting up the required libraries and environment for TextCNN sentiment analysis.

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

#  Basic Python libraries
import math
import numpy as np
import pandas as pd
import os
import math
import random
import codecs
from pathlib import Path

# MindSpore framework
import mindspore
import mindspore.dataset as ds
import mindspore.nn as nn
from mindspore import Tensor
from mindspore import context

# Training components
from mindspore.train.model import Model
from mindspore.nn.metrics import Accuracy
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor, TimeMonitor
from mindspore.ops import operations as ops

# Configuration utilities
from easydict import EasyDict as edict # Used to create a dictionary with attribute-style access

In [2]:
# Configuration parameters for TextCNN sentiment analysis
cfg = edict({
    # Project configuration
    'name': 'movie review',
    'pre_trained': False,
    'num_classes': 2,
    
    # Training parameters
    'batch_size': 64,
    'epoch_size': 4,
    'weight_decay': 3e-5,
    
    # Data configuration
    'data_path': './data/',
    'word_len': 51,
    'vec_length': 40,
    
    # Device and checkpoint configuration
    'device_target': 'CPU',
    'device_id': 0,
    'keep_checkpoint_max': 1,
    'checkpoint_path': 'ckpt/train_textcnn-4_596.ckpt',
})

# Set MindSpore execution context
context.set_context(mode=context.GRAPH_MODE, device_target=cfg.device_target, 
                    device_id=cfg.device_id)

## 📊 Dataset Download and Preprocessing
Processing the rt-polarity dataset for sentiment analysis with TextCNN.

In [3]:
# Read data.
# Read negative reviews
with open("./data/rt-polarity.neg", 'r', encoding='utf-8') as f:
    print("Negative reivews:")
    for i in range(5):
        print("[{0}]:{1}".format(i,f.readline()))

# Read positive reviews
with open("./data/rt-polarity.pos", 'r', encoding='utf-8') as f:
    print("Positive reivews:")
    for i in range(5):
        print("[{0}]:{1}".format(i,f.readline()))

Negative reivews:
[0]:simplistic , silly and tedious . 

[1]:it's so laddish and juvenile , only teenage boys could possibly find it funny . 

[2]:exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . 

[3]:[garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation . 

[4]:a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification . 

Positive reivews:
[0]:the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 

[1]:the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth . 

[2]:effective but too-tepid biopic

In [4]:
# Define the data generation class.
class Generator():
    def __init__(self, input_list):
        self.input_list=input_list
    
    def __getitem__(self,item):
        return (np.array(self.input_list[item][0],dtype=np.int32),
                np.array(self.input_list[item][1],dtype=np.int32))
    
    def __len__(self):
        return len(self.input_list)

In [5]:
class MovieReview:
    '''
    Movie review dataset
    '''
    def __init__(self, root_dir, maxlen, split):
        '''
        input:
        root_dir: movie review data directory
        maxlen: maximum length of a sentence
        split: the training/validation ratio in the dataset
        '''
        self.path = root_dir
        self.feelMap = {
            'neg':0,
            'pos':1
        }
        self.files = []
        self.doConvert = False
        
        mypath = Path(self.path)
        if not mypath.exists() or not mypath.is_dir():
            print("please check the root_dir!")
            raise ValueError
        
        # Find the files in the data directory.
        # Check whether there are two files: .neg and .pos.
        for root, dirs, files in os.walk(self.path):
            for file in files:
                if file.endswith('.neg') or file.endswith('.pos'):
                    self.files.append(os.path.join(root, file))
        
        # Read data.
        self.word_num = 0
        self.maxlen = 0
        self.minlen = float("inf")
        self.maxlen = float("-inf")
        self.Pos = []
        self.Neg = []
        
        for filename in self.files:
            self.read_data(filename)
        
        self.text2vec(maxlen=maxlen)
        self.split_dataset(split=split)

    def read_data(self, filePath):
        """
        Data reading and preprocessing method
        """
        with open(filePath,'r') as f:
            for sentence in f.readlines():
                # Text cleaning - remove punctuation and special characters
                sentence = sentence.replace('\n','')\
                    .replace('"','')\
                    .replace('\'','')\
                    .replace('.','')\
                    .replace(',','')\
                    .replace('[','')\
                    .replace(']','')\
                    .replace('(','')\
                    .replace(')','')\
                    .replace(':','')\
                    .replace('--','')\
                    .replace('-',' ')\
                    .replace('\\','')\
                    .replace('0','')\
                    .replace('1','')\
                    .replace('2','')\
                    .replace('3','')\
                    .replace('4','')\
                    .replace('5','')\
                    .replace('6','')\
                    .replace('7','')\
                    .replace('8','')\
                    .replace('9','')\
                    .replace('`','')\
                    .replace('=','')\
                    .replace('$','')\
                    .replace('/','')\
                    .replace('*','')\
                    .replace(';','')\
                    .replace('<b>','')\
                    .replace('%','')
                
                # Tokenization and filtering
                sentence = sentence.split(' ')
                sentence = list(filter(lambda x: x, sentence))
                
                # Statistics and classification
                if sentence:
                    self.word_num += len(sentence)
                    self.maxlen = self.maxlen if self.maxlen >= len(sentence) else len(sentence)
                    self.minlen = self.minlen if self.minlen <= len(sentence) else len(sentence)
                    
                    if 'pos' in filePath:
                        self.Pos.append([sentence,self.feelMap['pos']])
                    else:
                        self.Neg.append([sentence,self.feelMap['neg']])

    def text2vec(self, maxlen):
        """
        Text vectorization method - Convert sentences into vectors.
        """
        # Create vocabulary dictionary
        # Vocab = {word : index}
        self.Vocab = {'None': 0}
        vocab_index = 1
        
        # Build vocabulary from all sentences
        for SentenceLabel in self.Pos+self.Neg:
            for word in SentenceLabel[0]:
                if word not in self.Vocab.keys():
                    self.Vocab[word] = vocab_index
                    vocab_index += 1
        
        # Convert sentences to vectors
        for SentenceLabel in self.Pos+self.Neg:
            vector = [0]*maxlen
            for index, word in enumerate(SentenceLabel[0]):
                if index >= maxlen:
                    break
                if word not in self.Vocab.keys():
                    vector[index] = 0  # self.Vocab['None']
                else:
                    vector[index] = self.Vocab[word]
            SentenceLabel[0] = vector
        
        # Mark conversion as complete
        self.doConvert = True

    def split_dataset(self, split):
        """
        Dataset splitting method - Divide the dataset into a training set and a test set.
        """
        # Calculate split sizes
        trunk_pos_size = math.ceil((1-split)*len(self.Pos))
        trunk_neg_size = math.ceil((1-split)*len(self.Neg))
        trunk_num = int(1/(1-split))
        
        # Create temporary lists for splitting
        pos_temp=list()
        neg_temp=list()
        
        # Split data into chunks
        for index in range(trunk_num):
            pos_temp.append(self.Pos[index*trunk_pos_size:(index+1)*trunk_pos_size])
            neg_temp.append(self.Neg[index*trunk_neg_size:(index+1)*trunk_neg_size])
        
        # Create test and train sets
        self.test = pos_temp.pop(2)+neg_temp.pop(2)
        self.train = [i for item in pos_temp+neg_temp for i in item]
        random.shuffle(self.train)

    def get_dict_len(self):
        """
        Vocabulary dictionary utility method - Obtain the length of a dictionary consisting of characters in a dataset.
        """
        # Check if text vectorization is complete
        if self.doConvert:
            return len(self.Vocab)
        else:
            print("Haven't finished Text2Vec")
            return -1

    def create_train_dataset(self, epoch_size, batch_size):
        """
        Training dataset creation method
        """
        # Create generator dataset for training
        dataset = ds.GeneratorDataset(
            source=Generator(input_list=self.train), 
            column_names=["data","label"], 
            shuffle=False
        )
        
        # Apply batching and repetition for training
        dataset=dataset.batch(batch_size=batch_size,drop_remainder=True)
        dataset=dataset.repeat(epoch_size)
        return dataset

    def create_test_dataset(self, batch_size):
        """
        Test dataset creation method
        """
        # Create generator dataset for testing
        dataset = ds.GeneratorDataset(
            source=Generator(input_list=self.test), 
            column_names=["data","label"], 
            shuffle=False
        )
        
        # Apply batching for testing
        dataset=dataset.batch(batch_size=batch_size,drop_remainder=True)
        return dataset

In [6]:
# Create MovieReview instance and prepare training dataset
instance = MovieReview(root_dir=cfg.data_path, maxlen=cfg.word_len, split=0.9)
dataset = instance.create_train_dataset(batch_size=cfg.batch_size, epoch_size=cfg.epoch_size)
batch_num = dataset.get_dataset_size()

In [7]:
# Display the data processing results
vocab_size = instance.get_dict_len()
print("vocab_size: {0}".format(vocab_size))

# Show sample data from the dataset
item = dataset.create_dict_iterator()
for i, data in enumerate(item):
    if i < 1:
        print(data)
        print(data['data'][1])
    else:
        break

vocab_size: 18849
{'data': Tensor(shape=[64, 51], dtype=Int32, value=
[[   16,   729, 11161 ...     0,     0,     0],
 [  365, 14030,   416 ...     0,     0,     0],
 [10430,    11, 10431 ...     0,     0,     0],
 ...
 [ 1771,   129,    69 ...     0,     0,     0],
 [  108,  6943,    20 ...     0,     0,     0],
 [   16, 13816,   248 ...     0,     0,     0]]), 'label': Tensor(shape=[64], dtype=Int32, value= [1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 
 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 
 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0])}
[  365 14030   416   146  5334    11 12482   248     1  3734    11  9241
    85    33    76  2723   974  1284   416     1  1391    33  8627     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0]
[  365 14030   416   146  5334    11 12482   248     1  3734    11  9241
   

In [8]:
# Configure learning rate schedule with warm-up, normal, and shrink phases
learning_rate = []

# Warm-up phase: gradually increase learning rate
warm_up = [1e-3 / math.floor(cfg.epoch_size / 5) * (i + 1) for _ in range(batch_num) 
           for i in range(math.floor(cfg.epoch_size / 5))]

# Shrink phase: gradually decrease learning rate
shrink = [1e-3 / (16 * (i + 1)) for _ in range(batch_num) 
          for i in range(math.floor(cfg.epoch_size * 3 / 5))]

# Normal phase: constant learning rate
normal_run = [1e-3 for _ in range(batch_num) for i in 
              range(cfg.epoch_size - math.floor(cfg.epoch_size / 5) 
                    - math.floor(cfg.epoch_size * 2 / 5))]

# Combine all phases
learning_rate = learning_rate + warm_up + normal_run + shrink

## 🏗️ TextCNN Architecture
Building the TextCNN network with multiple convolutional filters for text sentiment analysis.

In [9]:
# TextCNN Model Definition
# The following cells define the TextCNN architecture with helper functions

In [10]:
# Helper function for weight initialization
def _weight_variable(shape, factor=0.01):
    """Initialize weights with random values"""
    init_value = np.random.randn(*shape).astype(np.float32) * factor
    return Tensor(init_value)


# Helper function to create convolutional layers
def make_conv_layer(kernel_size):
    """Create a convolutional layer with specified kernel size"""
    weight_shape = (96, 1, *kernel_size)
    weight = _weight_variable(weight_shape)
    return nn.Conv2d(in_channels=1, out_channels=96, kernel_size=kernel_size, padding=1,
                     pad_mode="pad", weight_init=weight, has_bias=True)

In [11]:
# TextCNN model class definition
class TextCNN(nn.Cell):
    def __init__(self, vocab_len, word_len, num_classes, vec_length):
        super(TextCNN, self).__init__()
        self.vec_length = vec_length
        self.word_len = word_len
        self.num_classes = num_classes
        self.unsqueeze = ops.ExpandDims()
        self.embedding = nn.Embedding(vocab_len, self.vec_length, embedding_table='normal')
        self.slice = ops.Slice()
        self.layer1 = self.make_layer(kernel_height=3)
        self.layer2 = self.make_layer(kernel_height=4)
        self.layer3 = self.make_layer(kernel_height=5)
        self.concat = ops.Concat(1)
        self.fc = nn.Dense(96*3, self.num_classes)
        self.drop = nn.Dropout(keep_prob=0.5)
        self.print = ops.Print()
        self.reducemean = ops.ReduceMax(keep_dims=False)
        
    def make_layer(self, kernel_height):
        return nn.SequentialCell(
            [
                make_conv_layer((kernel_height,self.vec_length)),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(self.word_len-kernel_height+1,1)),
            ]
        )
    
    def construct(self, x):
        """
        Forward pass of the TextCNN model.
        
        Args:
            x (Tensor): Input tensor with shape (batch_size, word_len)
            
        Returns:
            Tensor: Output logits for classification
        """
        # Convert input to embeddings
        x = self.embedding(x)  # Shape: (batch_size, word_len, vec_length)
        
        # Add channel dimension for convolution
        x = self.unsqueeze(x, 1)  # Shape: (batch_size, 1, word_len, vec_length)
        
        # Apply different convolutional layers with different kernel sizes
        x1 = self.layer1(x)  # Shape: (batch_size, 96, 1, 1)
        x2 = self.layer2(x)  # Shape: (batch_size, 96, 1, 1)
        x3 = self.layer3(x)  # Shape: (batch_size, 96, 1, 1)
        
        # Flatten the feature maps
        x1 = self.reducemean(x1, (2, 3))  # Shape: (batch_size, 96)
        x2 = self.reducemean(x2, (2, 3))  # Shape: (batch_size, 96)
        x3 = self.reducemean(x3, (2, 3))  # Shape: (batch_size, 96)
        
        # Concatenate features from all convolutional layers
        x = self.concat((x1, x2, x3))  # Shape: (batch_size, 288)
        
        # Apply dropout for regularization
        x = self.drop(x)
        
        # Final classification layer
        x = self.fc(x)  # Shape: (batch_size, num_classes)
        
        return x

In [12]:
# Create TextCNN model instance with vocabulary and configuration parameters
net = TextCNN(vocab_len=instance.get_dict_len(), 
              word_len=cfg.word_len, 
              num_classes=cfg.num_classes, 
              vec_length=cfg.vec_length)

print("TextCNN Model Architecture:")
print(net)

[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:52:38.874.21 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:52:38.929.91 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:52:38.929.91 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.


TextCNN Model Architecture:
TextCNN<
  (embedding): Embedding<vocab_size=18849, embedding_size=40, use_one_hot=False, embedding_table=Parameter (name=embedding.embedding_table, shape=(18849, 40), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=None>
  (layer1): SequentialCell<
    (0): Conv2d<input_channels=1, output_channels=96, kernel_size=(3, 40), stride=(1, 1), pad_mode=pad, padding=1, dilation=(1, 1), group=1, has_bias=True, weight_init=[[[[-0.01620523  0.00430576  0.00627976 ...  0.01322129 -0.00171179
         0.00499074]
       [ 0.00504958 -0.00449969 -0.01056014 ... -0.00311921 -0.01817112
         0.01028888]
       [-0.00263585 -0.01258848 -0.01530088 ...  0.00794003  0.02201227
         0.00944341]]]
    
    
     [[[ 0.00044081 -0.00835892 -0.00865513 ...  0.01416665 -0.0018302
        -0.0018223 ]
       [ 0.0104546  -0.00342864 -0.00826981 ... -0.01911463 -0.0002123
        -0.00121759]
       [-0.02338507 -0.00255891  0.00063968 ...  0.01364763 -0.00033

## ⚙️ Loss Function and Optimizer Configuration
Setting up the training components for TextCNN optimization.

In [13]:
# Configure optimizer and loss function
opt = nn.Adam(params=net.trainable_params(), learning_rate=learning_rate, weight_decay=cfg.weight_decay)
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

# Create model with optimizer, loss, and metrics
model = Model(net, loss_fn=loss, optimizer=opt, metrics={'acc': Accuracy()})

# Configure checkpoint settings
config_ck = CheckpointConfig(save_checkpoint_steps=batch_num, keep_checkpoint_max=cfg.keep_checkpoint_max)
ckpt_save_dir = "./ckpt"

# Setup training callbacks
time_cb = TimeMonitor(data_size=batch_num)
ckpoint_cb = ModelCheckpoint(prefix="train_textcnn", directory=ckpt_save_dir, config=config_ck)
loss_cb = LossMonitor(per_print_times=batch_num)  # Print loss at the end of each epoch

## 📈 Model Training
Training the TextCNN network on the sentiment analysis dataset.

In [14]:
# Start model training with configured callbacks
print("Starting TextCNN training...")
model.train(cfg.epoch_size, dataset, callbacks=[time_cb, ckpoint_cb, loss_cb])
print("Training completed successfully!")

[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:52:38.148.257 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.


Starting TextCNN training...


[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:52:38.161.944 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:52:38.170.829 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:52:38.170.829 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:52:38.195.468 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:52:38.200.064 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:13022439140

epoch: 1 step: 596, loss is 0.05056581273674965
Train epoch time: 21509.459 ms, per step time: 36.090 ms
Train epoch time: 21509.459 ms, per step time: 36.090 ms
epoch: 2 step: 596, loss is 0.0029518890660256147
Train epoch time: 20455.461 ms, per step time: 34.321 ms
epoch: 2 step: 596, loss is 0.0029518890660256147
Train epoch time: 20455.461 ms, per step time: 34.321 ms
epoch: 3 step: 596, loss is 0.002310193609446287
Train epoch time: 18841.412 ms, per step time: 31.613 ms
epoch: 3 step: 596, loss is 0.002310193609446287
Train epoch time: 18841.412 ms, per step time: 31.613 ms
epoch: 4 step: 596, loss is 0.002011424396187067
Train epoch time: 18965.490 ms, per step time: 31.821 ms
Training completed successfully!
epoch: 4 step: 596, loss is 0.002011424396187067
Train epoch time: 18965.490 ms, per step time: 31.821 ms
Training completed successfully!


## 🎯 Model Validation and Testing
Validating the trained TextCNN model on the sentiment analysis test dataset.

In [15]:
# Load the trained model from checkpoint
param_dict = load_checkpoint(cfg.checkpoint_path)
load_param_into_net(net, param_dict)
print("Model loaded successfully!")

# Create test dataset for evaluation
test_dataset = instance.create_test_dataset(batch_size=cfg.batch_size)

# Evaluate the model on test set
print("Evaluating model on test dataset...")
result = model.eval(test_dataset, dataset_sink_mode=False)
print("Test accuracy: {:.4f}".format(result['acc']))

[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:53:57.963.223 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:53:57.970.669 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:53:57.970.669 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:53:57.975.811 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:53:57.975.811 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.


Model loaded successfully!
Evaluating model on test dataset...
Test accuracy: 0.7588
Test accuracy: 0.7588


In [16]:
# Text preprocessing function for inference
def preprocess(sentence):
    """
    Preprocess input text for sentiment analysis inference.
    
    Args:
        sentence (str): Raw input text to be processed
        
    Returns:
        list: Vectorized representation of the input text
    """
    # Convert to lowercase and strip whitespace
    sentence = sentence.lower().strip()
    
    # Remove punctuation and special characters (same as training preprocessing)
    sentence = sentence.replace('\n','')\
        .replace('"','')\
        .replace('\'','')\
        .replace('.','')\
        .replace(',','')\
        .replace('[','')\
        .replace(']','')\
        .replace('(','')\
        .replace(')','')\
        .replace(':','')\
        .replace('--','')\
        .replace('-',' ')\
        .replace('\\','')\
        .replace('0','')\
        .replace('1','')\
        .replace('2','')\
        .replace('3','')\
        .replace('4','')\
        .replace('5','')\
        .replace('6','')\
        .replace('7','')\
        .replace('8','')\
        .replace('9','')\
        .replace('`','')\
        .replace('=','')\
        .replace('$','')\
        .replace('/','')\
        .replace('*','')\
        .replace(';','')\
        .replace('<b>','')\
        .replace('%','')\
        .replace("  "," ")  # Replace double spaces with single space
    
    # Tokenize the sentence
    sentence = sentence.split(' ')
    
    # Convert tokens to vector representation
    maxlen = cfg.word_len
    vector = [0] * maxlen
    
    for index, word in enumerate(sentence):
        if index >= maxlen:
            break
        if word not in instance.Vocab.keys():
            print(f"'{word}' - The word does not appear in the dictionary.")
        else:
            vector[index] = instance.Vocab[word]
    
    return vector


# Inference function for sentiment prediction
def inference(review_en):
    """
    Perform sentiment analysis inference on input text.
    
    Args:
        review_en (str): Input review text in English
        
    Returns:
        None: Prints the sentiment prediction result
    """
    # Preprocess the input text
    review_en = preprocess(review_en)
    
    # Convert to tensor format for model input
    input_en = Tensor(np.array([review_en]).astype(np.int32))
    
    # Get model prediction
    output = net(input_en)
    
    # Interpret the prediction result
    if np.argmax(np.array(output[0])) == 1:
        print("Positive comments")
    else:
        print("Negative comments")

In [17]:
# Test the model with a sample review
review_en = "the movie is so boring"
print(f"Input review: '{review_en}'")
print("Prediction:")
inference(review_en)

[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:53:58.227.927 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:53:58.235.686 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(11479:130224391405696,MainProcess):2025-09-12-15:53:58.235.686 [mindspore/nn/layer/basic.py:199] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.


Input review: 'the movie is so boring'
Prediction:
Negative comments


[ERROR] CORE(11479,76703a974080,python):2025-09-12-15:53:58.278.697 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_11479/3869061871.py]
[ERROR] CORE(11479,76703a974080,python):2025-09-12-15:53:58.278.740 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_11479/3869061871.py]
[ERROR] CORE(11479,76703a974080,python):2025-09-12-15:53:58.278.770 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_11479/3869061871.py]
